In [1]:
import torch
import torch.nn as nn
import numpy as np
from torch.autograd import grad
from torch.autograd.functional import jacobian
from operators import Δ, div, D

In [2]:
var = torch.tensor([[1.0, 1.0],
                   [4.0, 3.0]], requires_grad=True)
t = torch.tensor([[2.0], 
                  [3.0]], requires_grad=True)

var_2 = torch.tensor([[1.0],
                   [3.0]], requires_grad=True)
t_2 = torch.tensor([[1.0], 
                  [2.0]], requires_grad=True)
def u(x, t):
    xt = torch.cat((x, t), dim=1)
    return (1/6)*torch.pow(xt[:, 0].unsqueeze(1), 3) + xt[:,-1].unsqueeze(1)

In [ ]:
spatial_net = nn.Sequential(
            nn.Linear(2, 4),
            nn.ELU(),
            nn.Linear(4, 4),
            nn.ELU(),
            nn.Linear(4, 1)
        )

In [ ]:
out = u(var, t)
out_2 = u(var_2, t_2)

In [ ]:
torch.sum(jacobian(spatial_net, var), dim=0)

In [ ]:
out = spatial_net(var)
div(out, var)

In [ ]:
print(div(out.unsqueeze(1), t))
print(div(out_2.unsqueeze(1), t_2))

In [ ]:
print(div(out.unsqueeze(1), var))
print(div(out_2.unsqueeze(1), var_2))

In [ ]:
print(Δ(out.unsqueeze(1), var))
print(Δ(out_2.unsqueeze(1), var_2))

In [ ]:
loss = torch.square(div(out, t) - Δ(out, var)).mean()

In [ ]:
loss.backward()

In [2]:
var_1 = torch.tensor([[1.0],
                      [4.0]], requires_grad=True)
var_2 = torch.tensor([[2.0],
                      [8.0]], requires_grad=True)
var = [var_1, var_2]
C = torch.tensor([[2.0, 4.0],
                  [1.0, 3.0]], requires_grad=True)
def u(x, t):
    xt = torch.cat((x, t), dim=1)
    return (1/6)*torch.pow(xt[:, 0].unsqueeze(1), 3) + xt[:,-1].unsqueeze(1)

In [3]:
us = u(var_1, var_2)
print(us)

tensor([[ 2.1667],
        [18.6667]], grad_fn=<AddBackward0>)


In [8]:
div(us, var_2)

tensor([[1.],
        [1.]], grad_fn=<AddBackward0>)

In [3]:
class Infix:
    def __init__(self, function):
        self.function = function
    def __ror__(self, other):
        return Infix(lambda x, self=self, other=other: self.function(other, x))
    def __or__(self, other):
        return self.function(other)
    def __rlshift__(self, other):
        return Infix(lambda x, self=self, other=other: self.function(other, x))
    def __rshift__(self, other):
        return self.function(other)
    def __call__(self, value1, value2):
        return self.function(value1, value2)

In [20]:
def m_func(x, y):
    if isinstance(x, list):
        x = torch.cat(x, dim=-1)   
        return torch.einsum("bi, ij -> bj", x, y)
    y = torch.cat(y, dim=-1)
    return torch.einsum("ij, bj -> bi", x, y)
        
def v_func(x, y):
    if isinstance(x, list):
        x = torch.cat(x, dim=-1)
    if isinstance(y, list):
        y = torch.cat(y, dim=-1)
    
    return torch.einsum("bi, bi -> b", x, y).unsqueeze(1)
    
        

In [28]:
mdot = Infix(m_func)
vdot = Infix(v_func)
var_1 = torch.tensor([[1.0],
                      [4.0],
                      [4.0]], requires_grad=True)
var_2 = torch.tensor([[2.0],
                      [8.0],
                      [3.0]], requires_grad=True)
var = [var_1, var_2]
C = Matrix([[2.0, 4.0],
            [1.0, 3.0]], requires_grad=True)

In [31]:
var |vdot| (C |mdot| var)

tensor([[ 24.],
        [384.],
        [119.]], grad_fn=<AliasBackward>)

In [18]:
sum((, 2))

SyntaxError: invalid syntax (<ipython-input-18-4794d02002b1>, line 1)